In [1]:
import os
from copy import deepcopy

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from Environment import Environment, make_one_hot, give_mapping
from Agent import Agent

In [2]:
def give_prev_steps(prev_steps_assist, steps):
    prev_steps_assist = [[0,0,0,0,-1,-1] for i in range(steps-1)]
    return prev_steps_assist

In [3]:
N = 20
batch_size = 5
n_epochs = 5
alpha = 0.0005
memory_len = 6

agent = Agent(gamma=0.99, alpha=alpha, gae_lambda=0.95, policy_clip=0.1,\
              batch_size=batch_size, N=N, n_epochs=n_epochs, memory_len = memory_len)


n_games = 15000
best_reached = 0
score_history = []

learn_iters = 0
avg_score = 0
n_steps = 0
max_steps = 40
reached = 0

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (Subtract)             (None, 2)            0           tf_op_layer_strided_slice[0][0]  
                                                                 tf_op_layer_strided_sl

In [4]:
for i in tqdm(range(n_games)):
    env = Environment()
    env_cell_mapping = give_mapping(env.cells)
    layout = env_cell_mapping[np.newaxis, :, :, np.newaxis]
    done = False
    episode_reward = 0
    start, dest = env.give_start_dest()
    user_state = [start[0], start[1], dest[0], dest[1]]
    prev_steps_assist = []
    prev_steps_assist = give_prev_steps(prev_steps_assist, memory_len)
    step = 0
    episode_reward = 0
    curr_loc = user_state[:2]
    target_loc = user_state[2:]
    
    while not done and step<max_steps:
        step+=1
        n_steps+=1
        user_action, user_prob, asst_action, asst_prob, vals, asst_state, asst_output_one_hot\
        = agent.choose_action(user_state, layout, prev_steps_assist)
        
        
        new_loc, reward, _, done = env.step(user_action, asst_action + 1, target_loc, curr_loc)
        
        if done:
            reached+=1
        
        agent.remember(user_state, asst_state, layout, asst_output_one_hot,\
                       user_action, user_prob, asst_action, asst_prob, vals, reward, done)
         
        user_state = new_loc + target_loc
        prev_steps_assist = asst_state[1:]
        curr_loc = new_loc
        episode_reward+=reward
        
        if n_steps%N==0:
            agent.learn()
            learn_iters+=1
            
    score_history.append(episode_reward)
    avg_score = np.mean(score_history[-50:])
    
    if reached>=best_reached:
        best_reached = min(reached, 90)
        agent.save_models()
    
    if i%100==0:
        max_steps-=1
        max_steps = max(max_steps, 6)
    
    if i%100==0:
        print(f'Episode: {i}; Score: {episode_reward}; Avg_score: {avg_score}; Timesteps: {n_steps}; Learning Steps: {learn_iters}; Reached: {reached}')
        reached = 0

  0%|                                                                             | 1/15000 [00:04<17:06:20,  4.11s/it]

Saving Models
Episode: 0; Score: -40; Avg_score: -40.0; Timesteps: 40; Learning Steps: 2; Reached: 0


  0%|                                                                             | 2/15000 [00:05<14:05:22,  3.38s/it]

Saving Models


  0%|                                                                             | 3/15000 [00:08<12:47:32,  3.07s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████ | 14803/15000 [1:45:02<01:08,  2.89it/s]

Episode: 14800; Score: 6; Avg_score: 5.64; Timesteps: 81085; Learning Steps: 4054; Reached: 95


 99%|█████████████████████████████████████████████████████████████████████████▌| 14900/15000 [1:45:37<00:28,  3.50it/s]

Episode: 14900; Score: 7; Avg_score: 5.08; Timesteps: 81534; Learning Steps: 4076; Reached: 89


100%|██████████████████████████████████████████████████████████████████████████| 15000/15000 [1:46:08<00:00,  2.36it/s]


In [5]:
distribution = [0, 0, 0, 0]
reached = 0
agent.load_models()
max_steps = 6

for i in tqdm(range(5000)):
#     print('----------------------------------------------------------------------------------')
    env = Environment()
    env_cell_mapping = give_mapping(env.cells)
    layout = env_cell_mapping[np.newaxis, :, :, np.newaxis]
    done = False
    episode_reward = 0
    start, dest = env.give_start_dest()
    user_state = [start[0], start[1], dest[0], dest[1]]
    prev_steps_assist = []
    prev_steps_assist = give_prev_steps(prev_steps_assist, memory_len)
    step = 0
    episode_reward = 0
    curr_loc = user_state[:2]
    target_loc = user_state[2:]
    
    while not done and step<max_steps:
#         print(user_state)
        step+=1
        n_steps+=1
        user_action, user_prob, asst_action, asst_prob, vals, asst_state, asst_output_one_hot\
        = agent.choose_action(user_state, layout, prev_steps_assist)
        
#         print(asst_prob)
        
        distribution[asst_action]+=1
        new_loc, reward, _, done = env.step(user_action, asst_action + 1, target_loc, curr_loc)
        
        if done:
            reached+=1
        
#         agent.remember(user_state, asst_state, layout, asst_output_one_hot,\
#                        user_action, user_prob, asst_action, asst_prob, vals, reward, done)
         
        user_state = new_loc + target_loc
        prev_steps_assist = asst_state[1:]
        curr_loc = new_loc
        episode_reward+=reward
        
print(reached)
print(distribution)

Loading Models


100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [07:14<00:00, 11.50it/s]

4675
[9544, 1, 5879, 5145]
